In [ ]:
# Package can be installed from pip
!pip3 install stvae

In [11]:
import os
import stvae

In [2]:
# If we're using gpu accelerator
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

First of all, download and preprocess data:

In [3]:
!mkdir mouse
dataset = stvae.datasets.MouseDataset(directory='mouse', download=True)
# to make metrics better let's filter genes by variance
N_TOP_GENES = 700
dataset = stvae.preprocessing.get_high_variance_genes(dataset, n_genes=N_TOP_GENES)

100%|██████████| 38157/38157 [00:08<00:00, 4282.75it/s]


100%|██████████| 42/42 [00:00<00:00, 111.23it/s]


100%|██████████| 42/42 [00:00<00:00, 127.02it/s]
Transforming to str index.
/home/mrowl/.local/lib/python3.6/site-packages/scanpy/preprocessing/_deprecated/highly_variable_genes.py:193: RuntimeWarning: invalid value encountered in greater_equal
  gene_subset = df['dispersion_norm'].values >= disp_cut_off


In [4]:
# split data
train, test, classif = dataset.split(0.15, True, 0.15)

In [5]:
import torch

# the main way to set option: Use stvae.Config dataclass
cfg = stvae.Config()
cfg.count_labels = dataset.n_labels
cfg.count_classes = dataset.n_batches
cfg.use_cuda = torch.cuda.is_available()
cfg.input_dim = N_TOP_GENES
cfg.num_workers = None # If you want to limit CPU consumption, limit the number of workers (1/2)
model = stvae.stVAE(cfg)

In [6]:
cfg.verbose = 'none' # output
model.train(train, validation_data=None)

Experiment started: tmptgscqa52


Experiment tmptgscqa52 has finished


In [ ]:
metrics = model.test(test, classif)

In [8]:
metrics

{'MSE': 0.19630096852779388,
 'Calibration accuracy': 0.8779174147217235,
 'Cell type prediction accuracy [train]:': 0.989451476793249,
 'Cell type prediction accuracy expression [test hold out data]': 0.7737881508078994,
 'Cell type prediction accuracy expression [reconstructed data]': 0.7223219628964692,
 'Cell type prediction accuracy latents [reconstructed data]': 0.05565529622980251,
 'Cell type prediction accuracy expression [transfered to another classes]': 0.6983842010771992,
 'Cell type prediction accuracy latents [transfered to another classes]': 0.05565529622980251,
 'Class prediction accuracy': 0.7767803710353082}

Also we can use scanpy.Anndata, raw numpy array or scvi.dataset classes:

In [9]:
from scvi.dataset import RetinaDataset

In [10]:
dataset = RetinaDataset()
dataset = stvae.preprocessing.get_high_variance_genes(dataset, n_genes=N_TOP_GENES)

[2020-04-21 21:41:27,129] INFO - scvi.dataset.dataset | File /storage1/mrowl/stvae/data/retina.loom already downloaded
[2020-04-21 21:41:27,131] INFO - scvi.dataset.loom | Preprocessing dataset
[2020-04-21 21:41:35,378] INFO - scvi.dataset.loom | Finished preprocessing dataset
[2020-04-21 21:41:37,690] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-04-21 21:41:37,693] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
Transforming to str index.


In [25]:
model.train(train, validation_data=None)

Experiment started: tmpk5m_0o7q


Experiment tmpk5m_0o7q has finished


In [ ]:
# Test is same
metrics = model.test(test, classif)
print(metrics)